### Fetch the data

I' going to use the free API from [sunrisesunset.io](https://sunrisesunset.io/api/) to fetch the sunrise/sunset times for the current year. 

In [1]:
import requests as r

from dataclasses import dataclass
from datetime import datetime, date, timedelta

@dataclass 
class Location: 
    lat: float
    lon: float 

@dataclass
class DayDuration: 
    date: date
    duration: timedelta

    def as_hours(self):
        return self.duration.seconds / 60 / 60


Helsinki = Location(lat=60.192059, lon=24.945831)


/Users/sergeyhome/Projects/github/days-of-light/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
input_date_format = '%Y-%m-%d'
input_time_format = '%H:%M:%S'

def parse_api_date_object(obj): 
    parsed_date = datetime.strptime(obj['date'], input_date_format).date()
    day_length = datetime.strptime(obj['day_length'], input_time_format).time()
    day_duration = datetime.combine(date.min, day_length) - datetime.min
    return DayDuration(parsed_date, day_duration)


In [3]:
def get_year_bounds(): 
    now = datetime.now()
    start_of_year = date(now.year, 1, 1)
    end_of_year = date(now.year, 12, 31)
    return start_of_year, end_of_year

def get_request_url(location): 
    date_start, date_end = get_year_bounds()
    return "https://api.sunrisesunset.io/json?lat={}&lng={}&date_start={}&date_end={}".format(location.lat, location.lon, date_start, date_end)

def request_day_length_data(location): 
    response = r.get(get_request_url(location))
    assert response.status_code == 200

    json = response.json()
    assert json['status'] == 'OK'
    return [parse_api_date_object(obj) for obj in json['results']] 


In [4]:
data = request_day_length_data(Helsinki)

for entry in data: 
    print(entry.date, entry.as_hours())

2024-01-01 5.955555555555555
2024-01-02 5.986944444444444
2024-01-03 6.020833333333333
2024-01-04 6.057777777777777
2024-01-05 6.097222222222222
2024-01-06 6.139166666666667
2024-01-07 6.183888888888889
2024-01-08 6.230833333333334
2024-01-09 6.28
2024-01-10 6.331666666666666
2024-01-11 6.385277777777778
2024-01-12 6.441111111111111
2024-01-13 6.4991666666666665
2024-01-14 6.558888888888889
2024-01-15 6.620555555555556
2024-01-16 6.684166666666667
2024-01-17 6.749444444444444
2024-01-18 6.816111111111111
2024-01-19 6.884722222222222
2024-01-20 6.954444444444444
2024-01-21 7.025833333333334
2024-01-22 7.098611111111111
2024-01-23 7.1725
2024-01-24 7.2475000000000005
2024-01-25 7.323888888888889
2024-01-26 7.401388888888889
2024-01-27 7.479722222222223
2024-01-28 7.559166666666667
2024-01-29 7.639444444444445
2024-01-30 7.720833333333333
2024-01-31 7.802777777777778
2024-02-01 7.885555555555555
2024-02-02 7.969166666666666
2024-02-03 8.053333333333333
2024-02-04 8.138055555555557
2024-02